In [1]:
import Query
import Docket_OverView
import Docket_integration
import pandas as pd
import scipy.stats as stats
import os

In [2]:
output_dir = "../Data_input_for_BRCA/"
if os.path.isdir(output_dir) == False:
    os.mkdir(output_dir)

In [3]:
Query_WES = """
select COSMIC_identifier, HGNC_gene_symbol , AA_change,  Variant_Classification from `isb-cgc-04-0002.GDSC_v0.WES_variants` 
where COSMIC_identifier in 
(select COSMIC_identifier from `isb-cgc-04-0002.GDSC_v0.Cell_Line_Details`
where TCGA_Cancer_Type = 'BRCA' and Whole_Exome_Sequencing = 'Y' )
"""

mut = Query.Query_FromGCloud(Query_WES)
mut.to_csv(output_dir+"GDSC_MUT.csv")

In [4]:
Query_Mut_TCGA = """
select sample_barcode_tumor,Hugo_Symbol, HGVSp_Short,Variant_Type 
from `pancancer-atlas.Annotated.mc3_v0_2_8_PUBLIC_maf`
where project_short_name = 'TCGA-BRCA' and Variant_Classification in ('Frame_Shift_Ins','Nonsense_Mutation','Translation_Start_Site','In_Frame_Ins','Missense_Mutation','Splice_Site','Nonstop_Mutation')
"""
Mut_TCGA = Query.Query_FromGCloud(Query_Mut_TCGA)
Mut_TCGA.to_csv(output_dir+ "TCGA_mut.csv")

In [5]:
Query_Drug = """
select COSMIC_identifier, Cell_Line_Name, Drug_identifier, IC50, AUC from `isb-cgc-04-0002.GDSC_v0.Drug_Response`
where COSMIC_identifier in 
(select COSMIC_identifier from `isb-cgc-04-0002.GDSC_v0.Cell_Line_Details`
where TCGA_Cancer_Type = 'BRCA' and Whole_Exome_Sequencing = 'Y' and Drug_Response = 'Y')
"""

drug = Query.Query_FromGCloud(Query_Drug)
drug.to_csv(output_dir+ "GDSC_drug.csv")

In [6]:
query_EXP = '''
select COSMIC_identifier, gene_symbol, RMA_proc_basalExp from `isb-cgc-04-0002.GDSC_v0.Basal_Gene_Exp`
where COSMIC_identifier in 
(select COSMIC_identifier from `isb-cgc-04-0002.GDSC_v0.Cell_Line_Details`
where TCGA_Cancer_Type = 'BRCA' and Whole_Exome_Sequencing = 'Y' )
'''

Expr = Query.Query_FromGCloud(query_EXP)
Expr.to_csv(output_dir+"GDSC_Expr.csv")

In [7]:
Mut_TCGA['new_id'] = Mut_TCGA['Hugo_Symbol'] + Mut_TCGA['HGVSp_Short']
mut['new_id'] = mut['HGNC_gene_symbol'] + mut['AA_change']


In [8]:
mut.to_csv(output_dir+"Mut_site_GDSC.csv")
Mut_TCGA.to_csv(output_dir+ "Mut_site_TCGA.csv")

In [9]:
Mut_TCGA_matrix = Docket_integration.generate_mutation_matrix(Mut_TCGA,"sample_barcode_tumor","Hugo_Symbol","")
Mut_TCGA_matrix.to_csv(output_dir+ "Mut_TCGA_matrix.csv")

In [10]:
Mut_GDSC_matrix = Docket_integration.generate_mutation_matrix(mut,"COSMIC_identifier","HGNC_gene_symbol","")
Mut_GDSC_matrix.to_csv(output_dir+ "Mut_GDSC_matrix.csv")

In [11]:
Drug_GDSC_matrix = Docket_integration.generate_countious_matrix(drug,"COSMIC_identifier","Drug_identifier","IC50","")
Drug_GDSC_matrix.to_csv(output_dir+ "Drug_GDSC_matrix.csv")

In [12]:
Exp_GDSC_matrix = Docket_integration.generate_countious_matrix(Expr,"COSMIC_identifier","gene_symbol","RMA_proc_basalExp","Exp")
Exp_GDSC_matrix.to_csv(output_dir + "Exp_GDSC_matrix.csv" )